In [1]:
import numpy as np
from collections import deque
from copy import deepcopy

# Part 1 and 2

In [2]:
test = ['#.######',
        '#>>.<^<#',
        '#.<..<<#',
        '#>v.><>#',
        '#<^v^^>#',
        '######.#']

In [3]:
def build_graph(data):
    graph = {}
    for i in range(0, len(data)):
        data[i] = data[i].rstrip()
        for j in range(0, len(data[i])):
            if i == 0 and data[i][j] == '.':
                start = (i,j)
            elif i == len(data)-1 and data[i][j] == '.':
                end = (i,j)
            elif data[i][j] == '.':
                continue
            elif data[i][j] not in graph.keys():
                graph[data[i][j]] = np.array([[i,j]])
            else:
                graph[data[i][j]] = np.append(graph[data[i][j]], [[i,j]], axis=0)
    return graph, start, end

def run_blizzard(graph):
    max_x = np.max(graph['#'][:,1])
    
    graph['>'][:,1] += 1
    clip = np.where(graph['>'][:,1] == max_x)[0]
    graph['>'][clip,1] = 1
    
    graph['<'][:,1] -= 1
    clip = np.where(graph['<'][:,1] == 0)[0]
    graph['<'][clip,1] = max_x-1
    
    max_y = np.max(graph['#'][:,0])
    
    graph['v'][:,0] += 1
    clip = np.where(graph['v'][:,0] == max_y)[0]
    graph['v'][clip,0] = 1
    
    graph['^'][:,0] -= 1
    clip = np.where(graph['^'][:,0] == 0)[0]
    graph['^'][clip,0] = max_y-1
    
    return graph

def good_pos(graph, pos):
    if pos[1] < 0 or pos[2] < 0:
        return False
    if pos[1] >= len(graph) or pos[2] >= len(graph[pos[1]]):
        return False
    return graph[pos[1],pos[2]]

def print_graph(graph):
    max_i = np.max(graph['#'][:,0])
    max_j = np.max(graph['#'][:,1])
    for i in range(0, max_i+1):
        string = ''
        for j in range(0, max_j+1):
            c = ''
            for key in graph.keys():
                if (i,j) in list(map(tuple, graph[key])):
                    if len(c) == 0:
                        c = key
                    elif c in keys:
                        c = '2'
                    else:
                        c = str(int(c)+1)
            if len(c) == 0:
                c = '.'
            string += c
        print(string)
    print()
    return

def get_lcm(graph):
    max_i = np.max(graph['#'][:,0])-1
    max_j = np.max(graph['#'][:,1])-1
    return max_i, max_j, np.lcm(max_i, max_j)

def build_bool_graph(graph, max_i, max_j):
    new = np.ones((max_i+2, max_j+2), dtype=bool)
    for i in range(0, len(new)):
        for j in range(0, len(new[i])):
            pos = (i,j)
            for key in graph.keys():
                if pos in list(map(tuple, graph[key])):
                    new[i,j] = False
                    break
    return new

def build_graph_cube(graph):
    max_i, max_j, lcm = get_lcm(graph)
    bool_graph = np.array([build_bool_graph(graph, max_i, max_j)])
    #print_bool_graph(bool_graph[-1])
    for i in range(1, lcm):
        if i%(lcm//10) == 0:
            print('On', i, 'of', lcm)
        graph = run_blizzard(deepcopy(graph))
        bool_graph = np.append(bool_graph, [build_bool_graph(graph, max_i, max_j)], axis=0)
        #print_bool_graph(bool_graph[-1])
    return bool_graph, lcm

def print_bool_graph(graph):
    for i in range(0, len(graph)):
        string = ''
        for j in range(0, len(graph[i])):
            if graph[i][j]:
                string += '.'
            else:
                string += '#'
        print(string)
    print()
    return
        
#Breadth-First Search
def BFS(graph, start, end, lcm):
    dxy = [[1,0,0], [1,-1,0], [1,1,0], [1,0,-1], [1,0,1]]
    queue = deque([start])
    dist = {start: start[0]}
    
    while len(queue):
        cur_pos = queue.popleft()
        
        if (cur_pos[1], cur_pos[2]) == end:
            return cur_pos
        
        for i in range(0, 5):
            nxt_pos = (cur_pos[0]+dxy[i][0], cur_pos[1]+dxy[i][1], cur_pos[2]+dxy[i][2])
            if nxt_pos not in dist.keys() and good_pos(graph[nxt_pos[0]%lcm], nxt_pos):
                queue.append(nxt_pos)
                dist[nxt_pos] = nxt_pos[0]
                
    return -1

def run(data):
    graph, start, end = build_graph(data)
    start = (0, start[0], start[1])
    graph, lcm = build_graph_cube(graph)
    print('graph built, running BFS')
    time = BFS(graph, start, end, lcm)
    print('Part 1 result:', time[0])
    time = BFS(graph, time, (start[1],start[2]), lcm)
    time = BFS(graph, time, end, lcm)
    print('Part 2 result:', time[0])

In [4]:
run(test)

On 1 of 12
On 2 of 12
On 3 of 12
On 4 of 12
On 5 of 12
On 6 of 12
On 7 of 12
On 8 of 12
On 9 of 12
On 10 of 12
On 11 of 12
graph built, running BFS
Part 1 result: 18
Part 2 result: 54


In [5]:
with open('day24_input.txt', 'r') as f:
    inpt = f.readlines()
    f.close()
run(inpt)

On 60 of 600
On 120 of 600
On 180 of 600
On 240 of 600
On 300 of 600
On 360 of 600
On 420 of 600
On 480 of 600
On 540 of 600
graph built, running BFS
Part 1 result: 253
Part 2 result: 794
